In [2]:
# Imports
from jupyter_plotly_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from pymongo import MongoClient
import dash_leaflet as dl
import plotly.express as px
import dash_table
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from animal_shelter import AnimalShelter
import base64

###########################
# Data Manipulation / Model
###########################
# MongoDB autherization credentials for Python
username = "aacuser"
password = "1234"
shelter = AnimalShelter(username, password)

# Query MongoDB aac database via Python module and store results as dataframe object
# Dataframe object form Pandas/MongoDB
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('ModuleSeven')

# Load image from file
image_filename = 'Grazioso Salvare Logo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# HTML layout setup
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Window title
    html.Center(html.B(html.H4('Byron Laferriere: SNHU CS-340 Dashboard for Grazioso Salvare'))),
    # Customer image
    html.Center(html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),  
                         alt='customer image',style={'height':'10%', 'width':'10%'})),  
    # Client link
     html.Center(html.A('Navigate to our client site (snhu.edu)', href='https://www.snhu.edu', target='_blank')),
    # Dropdown selection object
    dcc.Dropdown(
        id='dropdown1',
        options=[
            {'label':'Water Rescue', 'value':'wr'},
            {'label':'Mountain or Wilderness', 'value':'mw'},
            {'label':'Disaster Rescue or Individual Tracking','value':'dr'},
            {'label':'Reset','value':'re'}
        ],
        value='Select...',
    ),

    # Dash table object setup
    dash_table.DataTable(
        id='datatable-id',
        data=df.to_dict('records'),
        # Set table column header names
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": False} for i in df.columns
        ],
        
        # Hides overflow of header text
        style_cell={
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
            'maxWidth': 300
        },
       
        editable=False,
        sort_action="native", # Enables sorting
        column_selectable=False, # Disables column select
        row_selectable=False, # Disables row select
        row_deletable=False, # Disables row delete
        page_action="native",
        page_current= 0, # Set current page
        page_size= 10, # Set row limit per page
                
    ),
    html.Br(),
    html.Hr(),
    # Set up map
    html.Div(
        id='map-id',
        style={'width':'50%', 'height':'500px','display':'inline-block','vertical-align':'top'}
    ),
    # Set up pie chart
    html.Div(
        dcc.Graph(id='graph', figure={}),
        style={'width':'35%', 'height':'500px','display':'inline-block','vertical-align':'top',  
               'text-align':'left'}
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
# Callback to update datatable 
@app.callback(Output('datatable-id','data'),
    [Input('dropdown1', 'value')
    ])
def update_output(value):
    # If dropdown is water rescue, update table with query results 
    if (value == 'wr'):
        dff = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",  
                "breed":{  
                    "$in":[
                        "Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"  
                    ]
                },  
                "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{  
                    "$gte":26  
                },  
                "age_upon_outcome_in_weeks":{  
                    "$lte":156  
                }
        }))) 
    # If dropdown is mountain or wilderness, update table with query results 
    elif (value == 'mw'):
        dff = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",  
                "breed":{  
                    "$in":[
                        "German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"  
                    ]
                },  
                "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{  
                    "$gte":26  
                },  
                "age_upon_outcome_in_weeks":{  
                    "$lte":156  
                }
        })))   
         # If dropdown is disaster rescue or individual tracking, update table with query results    
    elif (value == 'dr'):
        dff = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",  
                "breed":{  
                    "$in":[
                        "Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"  
                    ]
                },  
                "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{  
                    "$gte":20  
                },  
                "age_upon_outcome_in_weeks":{  
                    "$lte":300  
                }
        }))) 
    # If dropdown is reset, reset table to default      
    elif (value == 're'):
        dff = pd.DataFrame.from_records(shelter.read({}))
    return dff.to_dict('records')

# Callback to update Marker and pop-up data
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets lat and long
    temp_geo = "Lat: " + str(dff.iloc[0,13]) + ", Long: " + str(dff.iloc[0,14]) 
    # Sets id
    temp_id = "ID: " + str(dff.iloc[0,2]) + ", Type: " + str(dff.iloc[0,3]) 
    # Sets breeed
    temp_breed = "Breed: " + str(dff.iloc[0,4]) 
    return [
        # Geomap setup
        dl.Map(center=[dff.iloc[0,13],dff.iloc[0,14]], zoom=3.5, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(temp_geo),
                dl.Popup([
                    html.H1(temp_id),
                    html.P(temp_breed)
                ])
            ])
        ])
    ]
# Callback to update pie chart
@app.callback(
    Output(component_id='graph', component_property='figure'),
    [Input(component_id='dropdown1', component_property='value')]
)

def update_graph(value):
    dff=df
    # If dropdown is water rescue, update pie chart with query results 
    if (value == 'wr'):
        dff = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",  
                "breed":{  
                    "$in":[
                        "Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"  
                    ]
                },  
                "sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{  
                    "$gte":26  
                },  
                "age_upon_outcome_in_weeks":{  
                    "$lte":156  
                }
        }))) 
    # If dropdown is mountain or wilderness, update pie chart with query results
    elif (value == 'mw'):
        dff = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",  
                "breed":{  
                    "$in":[
                        "German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"  
                    ]
                },  
                "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{  
                    "$gte":26  
                },  
                "age_upon_outcome_in_weeks":{  
                    "$lte":156  
                }
        })))  
    # If dropdown is disaster rescue or individual tracking, update pie chart with query results  
    elif (value == 'dr'):
        dff = pd.DataFrame(list(shelter.read({
            "animal_type":"Dog",  
                "breed":{  
                    "$in":[
                        "Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"  
                        ]
                },  
                "sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{  
                    "$gte":20  
                },  
                "age_upon_outcome_in_weeks":{  
                    "$lte":300  
                }
        }))) 
    # If dropdown is reset, reset table to default 
    elif (value == 're'):
        dff = pd.DataFrame.from_records(shelter.read({}))
        
    piechart=px.pie(
        data_frame=dff,
        names='breed'
    )
    return (piechart)

app

In [10]:
 piechart = px.pie(
    data_frame = dff,
    names = 'breed'
    )
  return (piechart),

IndentationError: unexpected indent (<ipython-input-10-fb7436463038>, line 5)